In [4]:
import torch.nn as nn 
import torch 

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2)
        )
        
    def forward(self,x):
        return self.core(x)


policy = Model()
value = Model()

In [5]:
import gymnasium as gym 

env = gym.make("CartPole-v1",render_mode='human')


In [ ]:
def is_terminal(observation):
    x,v,theta , omega = observation 
    x_terminal = x <= -4.8 or x>=4.8
    theta_terminal = theta <= -0.209*7.5 or theta >= 0.209*7.5
    return x_terminal or theta_terminal


episodes = 1000
observations = []
rewards = []
actions = []

gamma = 0.99


for episode in range(episodes):
    done = False
    obs, info = env.reset(seed=42)
    obs = torch.tensor(obs)
    while not done:
        observations.append(obs)
        logits = policy(obs)
        probs = torch.softmax(logits,dim=-1)
        action = torch.multinomial(probs, num_samples=1).item()
        actions.append(action)
        obs, reward, terminated, truncated , info = env.step(action)
        rewards.append(reward)
        obs = torch.tensor(obs)
        done = is_terminal(obs)
    print('episode: ',episode + 1)
    

    discontinued_rewards = []
    reward = 0 
    for i in range(-1,-len(rewards),-1):
        reward = rewards[i] + gamma * reward
        discontinued_rewards.append(reward)





    

c:\Users\beca\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:214: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


episode:  1
episode:  2
episode:  3
episode:  4
episode:  5
episode:  6
episode:  7
episode:  8
episode:  9
episode:  10
episode:  11
episode:  12
episode:  13


KeyboardInterrupt: 